In [1]:
!pip install pyspark findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 49.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=bb8641ed231764e62709f94dc82c65b90c0cef0029e5f6850a08c69c37b56801
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

import math
import pandas as pd
import random
import numpy as np
from google.colab import drive 

import findspark

drive.mount('/content/drive')
findspark.init()
spark = SparkSession.builder.appName('SparkRDDExamples').getOrCreate()

Mounted at /content/drive


In [3]:
df_bibtex = spark.read.load('/content/drive/MyDrive/exercicio/results_bibtex.csv', format="csv", sep=",", inferSchema="true", header="true")

In [4]:
df_bibtex.show()

+---+--------------------+--------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|_c0|            keywords|numpages|         pages|          series|            location|            abstract|                 doi|                 url|             address|           publisher|                isbn|                year|              author|               title|    type_publication|                  id|              number|              volume|             journal|           booktitle|                issn|         note|              editor|             edition|
+---+--------------------+--------+---

In [5]:
df_scimago = spark.read.load('/content/drive/MyDrive/exercicio/scimagojr_2020.csv', format="csv", sep=";", inferSchema="true", header="true")

In [6]:
df_scimago.show()

+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|Rank|   Sourceid|               Title|   Type|              Issn|   SJR|SJR Best Quartile|H index|Total Docs. (2020)|Total Docs. (3years)|Total Refs.|Total Cites (3years)|Citable Docs. (3years)|Cites / Doc. (2years)|Ref. / Doc.|       Country|          Region|           Publisher|            Coverage|          Categories|
+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|   1|      28773|Ca-A Ca

In [7]:
df_jcs = spark.read.load('/content/drive/MyDrive/exercicio/jcs_2020.csv', format="csv", sep=";", inferSchema="true", header="true")

In [8]:
df_jcs.show()

+----+--------------------+-----------+----+---------------------+-----------------+----+----+
|Rank|  Full Journal Title|Total Cites| _c3|Journal Impact Factor|Eigenfactor Score| _c6| _c7|
+----+--------------------+-----------+----+---------------------+-----------------+----+----+
|   1|CA-A CANCER JOURN...|     55,868|null|              508.702|          0.10514|null|null|
|   2|NATURE REVIEWS MO...|     58,477|null|               94.444|          0.07548|null|null|
|   3|NEW ENGLAND JOURN...|    464,351|null|               91.245|          0.63118|null|null|
|   4|NATURE REVIEWS DR...|     41,989|null|               84.694|          0.04822|null|null|
|   5|              LANCET|    369,601|null|               79.321|          0.44524|null|null|
|   6|Nature Reviews Cl...|     17,973|null|               66.675|          0.03876|null|null|
|   7|Nature Reviews Ma...|     19,887|null|               66.308|          0.05677|null|null|
|   8|       Nature Energy|     28,166|null|      

In [9]:
df_jcs = df_jcs.drop("_c3")
df_jcs = df_jcs.drop("_c6")
df_jcs = df_jcs.drop("_c7")

In [10]:
df_scimago.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Sourceid: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Issn: string (nullable = true)
 |-- SJR: string (nullable = true)
 |-- SJR Best Quartile: string (nullable = true)
 |-- H index: integer (nullable = true)
 |-- Total Docs. (2020): integer (nullable = true)
 |-- Total Docs. (3years): integer (nullable = true)
 |-- Total Refs.: integer (nullable = true)
 |-- Total Cites (3years): integer (nullable = true)
 |-- Citable Docs. (3years): integer (nullable = true)
 |-- Cites / Doc. (2years): string (nullable = true)
 |-- Ref. / Doc.: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Coverage: string (nullable = true)
 |-- Categories: string (nullable = true)



In [11]:
for column in df_jcs.columns:
  df_jcs = df_jcs.withColumnRenamed(column, column.lower())

df_jcs = df_jcs.withColumnRenamed("Full Journal Title","title")


In [12]:
from pyspark.sql.functions import lower, col


df_jcs = df_jcs.select("*", lower(col("title")))
df_scimago = df_scimago.select("*", lower(col("title")))


In [13]:
for column in df_scimago.columns:
  df_scimago = df_scimago.withColumnRenamed(column, column.lower())


In [14]:
df_scimago = df_scimago.withColumnRenamed("lower(title)","lower_title")
df_jcs = df_jcs.withColumnRenamed("lower(title)","lower_title")

In [15]:
from pyspark.sql.functions import desc
df_ranks = df_scimago.join(df_jcs, df_scimago.lower_title == df_jcs.lower_title, 'inner')

In [16]:
df_ranks.show()

+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+-----------+---------------------+-----------------+--------------------+
|rank|   sourceid|               title|   type|              issn|   sjr|sjr best quartile|h index|total docs. (2020)|total docs. (3years)|total refs.|total cites (3years)|citable docs. (3years)|cites / doc. (2years)|ref. / doc.|       country|          region|           publisher|            coverage|          categories|         lower_title|rank|               title|total cites|journal impact factor|eigenfactor score|         lower_title|
+----+-----------+--------------------+-------+------------------+------+-----------------+-------+-----------

In [17]:
from pyspark.sql.functions import split, regexp_replace

In [18]:
df_ranks = df_ranks.withColumn('issn_0', split(df_ranks['issn'], ', ').getItem(0)) \
       .withColumn('issn_1', split(df_ranks['issn'], ', ').getItem(1)) \
       .withColumn('issn_2', split(df_ranks['issn'], ', ').getItem(2))

In [19]:
df_ranks.show()

+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+-----------+---------------------+-----------------+--------------------+--------+--------+------+
|rank|   sourceid|               title|   type|              issn|   sjr|sjr best quartile|h index|total docs. (2020)|total docs. (3years)|total refs.|total cites (3years)|citable docs. (3years)|cites / doc. (2years)|ref. / doc.|       country|          region|           publisher|            coverage|          categories|         lower_title|rank|               title|total cites|journal impact factor|eigenfactor score|         lower_title|  issn_0|  issn_1|issn_2|
+----+-----------+--------------------+-------+-------------

In [20]:
df_main = df_bibtex.withColumn('issn', regexp_replace(df_bibtex.issn, '-', '')) \

In [21]:
df_main.show()

+---+--------------------+--------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|_c0|            keywords|numpages|         pages|          series|            location|            abstract|                 doi|                 url|             address|           publisher|                isbn|                year|              author|               title|    type_publication|                  id|              number|              volume|             journal|           booktitle|                issn|         note|              editor|             edition|
+---+--------------------+--------+---

In [23]:
df_merge_1 = df_main.join(df_ranks, df_main.issn == df_ranks.issn_0, 'left')
df_merge_2 = df_main.join(df_ranks, df_main.issn == df_ranks.issn_1, 'left')
df_merge_3 = df_main.join(df_ranks, df_main.issn == df_ranks.issn_2, 'left')

In [24]:
df_total_0 = df_merge_1.union(df_merge_2)
df_total = df_total_0.union(df_merge_3)

# To remove the duplicates:

result = df_total.dropDuplicates()

In [25]:
result.show()

+---+--------------------+--------+------------+-----------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+----+--------------------+--------------------+----------------+--------------------+------+------+--------------------+--------------------+----+----+------+-------+----+--------+-----+----+----+----+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+-------+------+---------+--------+----------+-----------+----+-----+-----------+---------------------+-----------------+-----------+------+------+------+
|_c0|            keywords|numpages|       pages|     series|            location|            abstract|                 doi|                 url|          address|           publisher|         isbn|year|              author|               title|type_publication|                  id|number|vol